**This is the codebook about labelled multi-classification.  
The methods we used: Logistics, XGboost, new feature(word count) and oversample.      
Finally, the logitics without word count perform best, which obtain acc 0.63.**

**multi-classfication  
wordcount+logistics**

In [4]:
from sklearn import model_selection, preprocessing, linear_model, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_prep2 = pd.read_csv('df_prep2.csv')

In [5]:
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
df_prep2['word_count'] = [len(item.split(" ")) for item in df_prep2['contents']]
x_train, x_test, y_train, y_test = model_selection.train_test_split(df_prep2.contents,df_prep2['risk_cat'],
                                                                    test_size = 0.2, random_state = 42) 

y_train = y_train.replace(['Macro Environment', 'Product Availability', 'Not Risk',
       'Industry Adjacent', 'Invalid', 'Product Failure',
       'Service Failure/Availability', 'Others',
       'Changes in Company Policy', 'Employee Misconduct',
       'Customer Misconduct', 'Confrontation'],[0,1,2,3,4,5,6,7,8,9,10,11])
y_test = y_test.replace(['Macro Environment', 'Product Availability', 'Not Risk',
       'Industry Adjacent', 'Invalid', 'Product Failure',
       'Service Failure/Availability', 'Others',
       'Changes in Company Policy', 'Employee Misconduct',
       'Customer Misconduct', 'Confrontation'],[0,1,2,3,4,5,6,7,8,9,10,11])
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)
y_train['risk_cat'] = y_train['risk_cat'].astype('int')
y_test['risk_cat'] = y_test['risk_cat'].astype('int')

tfidf1 = TfidfVectorizer() 
tfidf1.fit(x_train)  
x_train= tfidf1.transform(x_train) 
x_test = tfidf1.transform(x_test) 
x_train_wc =  hstack([x_train,np.array([df_prep2.iloc[y_train.index.values]['word_count']]).T])
x_test_wc =  hstack([x_test,np.array([df_prep2.iloc[y_test.index.values]['word_count']]).T])

In [7]:
lr1 = LogisticRegression(multi_class='multinomial',random_state=123,penalty='l2', solver='lbfgs')
lr1.fit(x_train_wc, y_train)
y_pred_log = lr1.predict_proba(x_test_wc)

import numpy as np
y_pred_log = [np.argmax(line) for line in y_pred_log]
print('accuracy %s' % accuracy_score(y_pred_log, y_test))
print(classification_report(y_test, y_pred_log))

C:\Users\E055761\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


accuracy 0.41165234001910217
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.00      0.00      0.00        66
           2       0.43      0.83      0.56       244
           3       0.00      0.00      0.00        30
           4       0.00      0.00      0.00        26
           5       0.00      0.00      0.00        45
           6       0.43      0.63      0.51       124
           7       0.32      0.61      0.42       176
           8       0.00      0.00      0.00        97
           9       0.79      0.28      0.41       157
          10       0.00      0.00      0.00        21
          11       0.00      0.00      0.00        22

    accuracy                           0.41      1047
   macro avg       0.16      0.20      0.16      1047
weighted avg       0.32      0.41      0.32      1047



C:\Users\E055761\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\E055761\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**multi-classification  
wordcount+xgb**

In [74]:
import xgboost as xgb
dtrain = xgb.DMatrix(data=x_train, label=y_train)
dtest = xgb.DMatrix(data=x_test)
dtest = xgb.DMatrix(data=x_test)
param = {
    'max_depth': 6,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 12}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations
xg1 = xgb.train(param, dtrain, num_round)
pred1 = xg1.predict(dtest)
pred1 = [np.argmax(line) for line in pred1]
print('accuracy %s' % accuracy_score(pred1, y_test))
print(classification_report(y_test, pred1))

[16:54:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy 0.6074498567335244
              precision    recall  f1-score   support

           0       0.64      0.23      0.34        39
           1       0.64      0.56      0.60        66
           2       0.57      0.76      0.65       244
           3       0.75      0.20      0.32        30
           4       0.21      0.15      0.18        26
           5       0.93      0.93      0.93        45
           6       0.66      0.60      0.63       124
           7       0.52      0.63      0.57       176
           8       0.74      0.55      0.63        97
           9       0.64      0.62      0.63       157
          10       0.91      0.48      0.62        21
          11

**multi-classification  
without word count + logistics**

In [102]:
#without wordcount
x_train, x_test, y_train, y_test = model_selection.train_test_split(df_prep2.contents,df_prep2['risk_cat'],
                                                                    test_size = 0.2, random_state = 42) 
y_train = y_train.replace(['Macro Environment', 'Product Availability', 'Not Risk',
       'Industry Adjacent', 'Invalid', 'Product Failure',
       'Service Failure/Availability', 'Others',
       'Changes in Company Policy', 'Employee Misconduct',
       'Customer Misconduct', 'Confrontation'],[0,1,2,3,4,5,6,7,8,9,10,11])
y_test = y_test.replace(['Macro Environment', 'Product Availability', 'Not Risk',
       'Industry Adjacent', 'Invalid', 'Product Failure',
       'Service Failure/Availability', 'Others',
       'Changes in Company Policy', 'Employee Misconduct',
       'Customer Misconduct', 'Confrontation'],[0,1,2,3,4,5,6,7,8,9,10,11])
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)
y_train['risk_cat'] = y_train['risk_cat'].astype('int')
y_test['risk_cat'] = y_test['risk_cat'].astype('int')

tfidf1 = TfidfVectorizer() 
tfidf1.fit(x_train)  
x_train= tfidf1.transform(x_train) 
x_test = tfidf1.transform(x_test) 


lr1 = LogisticRegression(multi_class='multinomial',random_state=123,penalty='l2', solver='lbfgs')
lr1.fit(x_train, y_train)
y_pred_log = lr1.predict_proba(x_test)


y_pred_log = [np.argmax(line) for line in y_pred_log]
print('accuracy %s' % accuracy_score(y_pred_log, y_test))
print(classification_report(y_test, y_pred_log))

accuracy 0.626552053486151
              precision    recall  f1-score   support

           0       0.81      0.33      0.47        39
           1       0.73      0.58      0.64        66
           2       0.55      0.80      0.65       244
           3       0.50      0.17      0.25        30
           4       1.00      0.08      0.14        26
           5       0.93      0.84      0.88        45
           6       0.68      0.65      0.66       124
           7       0.54      0.66      0.59       176
           8       0.76      0.60      0.67        97
           9       0.65      0.64      0.64       157
          10       1.00      0.48      0.65        21
          11       1.00      0.09      0.17        22

    accuracy                           0.63      1047
   macro avg       0.76      0.49      0.54      1047
weighted avg       0.66      0.63      0.61      1047



**multi-classification  
without word count + xgb**

In [105]:
dtrain = xgb.DMatrix(data=x_train, label=y_train)
dtest = xgb.DMatrix(data=x_test)
dtest = xgb.DMatrix(data=x_test)
param = {
    'max_depth': 6,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 12}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations
xg1 = xgb.train(param, dtrain, num_round)
pred1 = xg1.predict(dtest)
pred1 = [np.argmax(line) for line in pred1]
print('accuracy %s' % accuracy_score(pred1, y_test))
print(classification_report(y_test, pred1))

[17:34:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy 0.5959885386819485
              precision    recall  f1-score   support

           0       0.54      0.18      0.27        39
           1       0.61      0.55      0.58        66
           2       0.57      0.75      0.65       244
           3       0.60      0.20      0.30        30
           4       0.12      0.08      0.10        26
           5       0.91      0.93      0.92        45
           6       0.63      0.62      0.62       124
           7       0.50      0.62      0.55       176
           8       0.77      0.55      0.64        97
           9       0.64      0.59      0.61       157
          10       1.00      0.48      0.65        21
          11